In [8]:
import json
import imageio
import numpy as np
import os
import re
from skimage import draw, io
import uuid
import skimage.transform
import scipy
import warnings

In [9]:
# labelbox_json = json.load(open('./rocks.json'))
labelbox_json = json.load(open('geo_solution_rocks.json'))
output_path = './output'
# resize = True
resize = False

In [10]:
def resize_image_to_square(image, side_dim=1992):
    image_dtype = image.dtype
    
    h, w = image.shape[:2]
    window = (0, 0, h, w)
    padding = [(0, 0), (0, 0), (0, 0)]

    max_dim = max(h, w)

    top_pad = (max_dim - h) // 2
    bottom_pad = max_dim - h - top_pad
    left_pad = (max_dim - w) // 2
    right_pad = max_dim - w - left_pad
    padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
    image = np.pad(image, padding, mode='constant', constant_values=0)
    
    image = skimage.transform.resize(image, (side_dim, side_dim), order=1, mode="constant", preserve_range=True)
    
    print(padding)
    return image.astype(image_dtype), padding


In [11]:
def resize_image_to_square2(image, side_dim=1992):
    image_dtype = image.dtype
    
    h, w = image.shape[:2]
    window = (0, 0, h, w)
    padding = [(0, 0), (0, 0), (0, 0)]

    scale = 1
    image_max = max(h, w)
    if round(image_max * scale) > side_dim:
        scale = side_dim / image_max

    # Resize image using bilinear interpolation
    if scale != 1:
        image = skimage.transform.resize(image, (round(h * scale), round(w * scale)), order=1, mode="constant", preserve_range=True)
        h, w = image.shape[:2]
        
    top_pad = (side_dim - h) // 2
    bottom_pad = side_dim - h - top_pad
    left_pad = (side_dim - w) // 2
    right_pad = side_dim - w - left_pad
    padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
    image = np.pad(image, padding, mode='constant', constant_values=0)
    
    print(padding)
    return image.astype(image_dtype), padding, scale

In [12]:
def resize_mask(mask, padding):
    return np.pad(mask, padding, mode='constant', constant_values=0)

In [13]:

def resize_mask2(mask, padding, scale):
    # Suppress warning from scipy 0.13.0, the output shape of zoom() is
    # calculated with round() instead of int()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mask = scipy.ndimage.zoom(mask, zoom=[scale, scale], order=0)
        
    return np.pad(mask, padding, mode='constant', constant_values=0)

In [ ]:
for rock in labelbox_json:
    image_name = rock['External ID']
    image_url = rock['Labeled Data']
    print("image_name: {}".format(image_name))
    print("image_url: {}".format(image_url))
    
    image = io.imread(image_url)
    height, width = image.shape[:2]
    
    if resize:
#         image, padding = resize_image_to_square(image)
        image, padding, scale = resize_image_to_square2(image)
    
    labels = rock['Label']
    if labels == 'Skip':
        continue
        
    for label_name, label_polygons in labels.items():
        ## TODO: Add support to multiple labels and remove this check
        if not label_name == 'Vein':
            continue
            
        for index, label_polygon in enumerate(label_polygons):
            mask = np.zeros((height, width), dtype=np.uint8)
            all_points_x = []
            all_points_y = []
            for coords in label_polygon:
                all_points_x.append(coords['x'])
                all_points_y.append(coords['y'])

            rr, cc = draw.polygon(all_points_y, all_points_x)
            
            mask[rr, cc] = 1
            mask = np.flipud(mask)
            if resize:
#                 mask = resize_mask(mask, padding[:2])
                mask = resize_mask2(mask, padding[:2], scale)
                
            image_output_path = os.path.join(output_path, 'images', '{}.{}.png'.format(image_name, index))
            imageio.imwrite(image_output_path, image)
            
            instance_mask_output_path = os.path.join(output_path, 'masks', '{}.{}.png'.format(image_name, index))
            imageio.imwrite(instance_mask_output_path, mask)
#     break


image_name: NC548_0239_00.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F4f50bda7-fe24-4ab7-a7ad-0747f475dbae%2FNC548_0239_00.png?alt=media&token=fe8ca4e9-d475-4e00-9aa3-29f851ef67b2
image_name: NC548_0240_00.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F4f50bda7-fe24-4ab7-a7ad-0747f475dbae%2FNC548_0240_00.png?alt=media&token=effd3607-0e44-4921-afb8-d2a4e7548852
image_name: NC548_0239_01.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F4f50bda7-fe24-4ab7-a7ad-0747f475dbae%2FNC548_0239_01.png?alt=media&token=e0746f28-f59f-4ce4-8a5a-c63e8dfe7579
image_name: NC548_0239_02.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F4f50bda7-fe24-4ab7-a7ad-0747f475dbae%2FNC548_0239_02.png?alt=media&token=73ab98c8-5eba-4e66-aee9-b

image_name: UE118_0092_00.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2Fb6a92997-b661-41dd-96c1-347d90a6e878%2FUE118_0092_00.png?alt=media&token=4627daf4-03ac-44ce-a921-27ed465bafdf
image_name: UE118_0092_01.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2Fb6a92997-b661-41dd-96c1-347d90a6e878%2FUE118_0092_01.png?alt=media&token=fbefc910-554f-496b-85e3-c46559e2c9ad
image_name: UE118_0101_01.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2Fb6a92997-b661-41dd-96c1-347d90a6e878%2FUE118_0101_01.png?alt=media&token=77654552-8707-4166-8f9b-dccfbbca0b30
image_name: UE118_0101_02.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2Fb6a92997-b661-41dd-96c1-347d90a6e878%2FUE118_0101_02.png?alt=media&token=d16fd006-e895-40d7-8d02-7